# 循环神经网络

作者：杨岱川

时间：2020年03月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文献：

- 《深度学习》，作者：Ian Goodfellow 、Yoshua Bengio、Aaron Courville。
- [CS231n](http://cs231n.stanford.edu/)
- 《动手学深度学习》，作者：李沐、阿斯顿·张

## 本节目的

多层感知机和卷积神经网络能够有效处理空间信息，而循环神经网络是为了更好地处理时序信息而设计。循环神经网络引入状态变量来保存过去的信息，然后与当前的输入共同决定当前的输出。

本节我们会学习循环神经网络的原理和使用方法。

## 序列数据与语言模型

首先我们会举例说明什么事序列数据，然后简单讲述一下语言模型。

### 序列数据是什么

一段文字、一段声音，或者电影胶片中前后相接的画面顺序，都是序列数据。可以认为有一些时间顺序和规律存在其中。

拿文字来说明，有这么一句话：

> 今天的天气真好呀

这句话就能被看做是一段离散的时间序列数据。这句话共有 $T = 8$ 个字，按索引来说，每个字我们表示为 $w_1, w_2, \cdots, w_T$，自然有 

> $w_1 = 今， w_2 = 天， \cdots，w_8 = 呀$

这样的一个序列可以用概率来表示，如 $P(w_1,w_2,\cdots,w_T)$，称为序列的概率。

在深度学习领域，对于语言文字的处理，或者叫 **自然语言处理（NLP）** 是非常热门的方向之一。在这个领域处理的数据大多都是类似的序列数据。

### 语言模型

**语言模型（Language Model）** 是自然语言处理的重要技术。上文中序列的概率 $P(w_1,w_2,\cdots,w_T)$ 就是通过语言模型计算出来的。

语言模型的应用十分广泛，比如“谷歌翻译”、“百度翻译”或者是一些智能问答系统等产品中都会使用语言模型来建模。

比如在英文翻译中常见的笑话：

> “How old are you？”

如果字对字地翻译会成为：

> “怎么老是你？”

当然按句意翻译实际应该是：

> “你多大了？”

------------

语言模型会针对这样的翻译文本计算一个概率。如果算出 “你多大了” 这个翻译结果的概率 $P$ 最大，那么我们就将

> “How old are you？”

翻译为

> “你多大了？”。

### 语言模型的建立

语言模型会怎么计算概率呢？这里我们要给出一个假设：

> 在序列 $w_1, w_2, \cdots, w_T$ 中，每个字（每个词）是依次生成的。

就如我们看书和说话一般，说了上一个字才说下一个字。

所以我们有：

$$\large P(w_1, w_2, \cdots, w_T) = \prod_{t=1}^{T}P(w_t|w_1,\cdots,w_{t-1})$$

比如有这么一句话 $S$:

> $S = 天气真好$

则我们有：

> $w_1 = 天， w_2 = 气， w_3 = 真，w_4 = 好$

那么这句话的概率就是：

$$\large P(S)=P(w_1, w_2, w_3, w_4)=P(w_1)P(w_2|w_1)P(w_3|w_1,w_2)P(w_4|w_1,w_2,w_3)$$

需要说明的是，对于中文不一定是**按字分割**，传统做法更多是**按词分割**，比如 $w_1=天气$ ，$w_2=真$，$w_3=好$。

### 语言模型的计算

建立好语言模型以后，我们需要实际进行计算。

那么，$P(w_1=天)$ 到底等于多少呢？

假设我们有一个大型的文本语料库，比如百度百科或者别的什么数据集，我们可以从中计算出“天”字出现的次数占语料库总字数比例。

比如语料库总共有 10000 个字（实际通常多得多），天字有 33 个。那么 ：

$$\large P(w_1=天) = 0.0033$$

接下来是 $P(w_2|w_1)$：

> $P(w_2=气|w_1=天)$ ：第一个字是“天”的情况下，第二个字是“气”的概率。

所以我们要统计在语料库的 33 个天字后面，有多少跟着“气”字。

假定有 5 个“天气”吧，所以有：

$$\large P(w_2=气|w_1=天) = 0.1515$$

以此类推，我们可以知道 $P(w_3|w_1,w_2)$ 就是要计算 “天气” 后面跟着 “真” 字的概率。

最后根据上面的公式，我们就能得到 $P(w_1, w_2, w_3, w_4)$ 的概率了。

### n元语法

当序列长度非常长的时候，计算和储存多个字（词）的概率会非常非常复杂，所以人们发明了 **n元语法（n-gram grammar）** 来简化语言模型的计算。

> **马尔科夫假设（Markov Assumption）**：一个字（词）的出现只和前面 $n$ 个字（词）有关。

这也叫做 **$n$ 阶马尔科夫链**。

如果 $n=1$，那么有 $P(w_3|w_1,w_2)=P(w_3|w_2)$。

如果基于 $n-1$ 阶马尔科夫链，我们可以把语言模型改写为

$$\large(w_1,w_2,\cdots,w_T) \approx \prod_{t=1}^{T}P(w_t|w_{t-(n-1)},\cdots,w_{t-1})$$

特别说明，当 $n$ 分别为 1、2、3 的时候，我们将其称为一元语法（unigram）、二元语法（bigram）、三元语法（trigram）。

以二元语法为例，之前的例子可以化简为：

$$\large P(w_1, w_2, w_3, w_4)=P(w_1)P(w_2|w_1)P(w_3|w_2)P(w_4|w_3)$$

当然我们要明白：

>当 $n$ 较小的时候，通常并不准确；而当 $n$ 较大的时候，计算和存储又会相当复杂，参数数量呈指数级增长。

## 循环神经网络

为了解决 n元语法 的问题，我们会介绍**循环神经网络**。

循环神经网络并非刚性地几亿所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。

### 隐藏状态与展开

与普通的神经网络不同，循环神经网络增加了一个隐藏状态 $h$ 来记录之前时间步的信息。

> “时间步”不一定是字面上的现实世界中流逝的时间，有时候它仅仅表示序列中的位置。

我们可以用下式来表示隐藏状态的变化：

$$\large h^{(t)} = f \left( h^{(t-1)}, x^{(t)}; \theta \right)$$

其中，

- $h^{(t)}$ 表示 $t$ 时间步的隐藏状态；
- $x^{(t)}$ 表示 $t$ 时间步的输入；
- $\theta$ 表示函数的相关参数。

当然这么写可能不是特别明确，因为这个函数是一个**抽象函数**的写法。我们需要看着相关的结构图来理解。

首先我们看一个没有输出的循环网络：

![RNN1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/RNN1.png?raw=true)

在这个网络中，只处理来自输入 $x$ 的信息，然后将其合并到经过时间想去传播的状态 $h$ 。

图片左边是回路的**原理图**，方块表示单个时间步的延迟。

右边的同一网络被视为**展开的计算图**，其中每个节点现在与一个特定的时间实例相关联。

具体来说，原理图中的橙色方块表示时刻 $t$ 的状态到时刻 $t+1$ 的状态之间的相互作用，具体的数学表达式我们会在后面介绍。

而在展开图中，每个时间步的每个变量被绘制为图中的一个独立节点。至于展开图应该有多长，实际上取决于**序列的长度**。

我们用一个函数 $g^{(t)}$ 代表经过 $t$ 步展开后的循环：

$$
\large 
\begin{equation}\begin{split} 
h^{(t)} &= g^{(t)}\left(x^{(t)}, x^{(t-1)}, x^{(t-2)}, \cdots, x^{(2)}, x^{(1)} \right)\\
&= f\left( h^{(t-1)}, x^{(t)};\theta \right)
\end{split} \end{equation}$$
 
函数 $g^{(t)}$ 将全部的过去序列 $\left(x^{(t)}, x^{(t-1)}, x^{(t-2)}, \cdots, x^{(2)}, x^{(1)} \right)$ 作为输入来生成当前状态。在展开的计算图中，我们用更简单的方法，也就是吧 $g^{(t)}$ 分解为函数 $f$ 的重复应用。

展开过程的优点主要有两个：

- 无论序列的长度，学成的模型始终具有相同的输入大小，即上个时间步的隐藏状态和当前时间步的输入
- 我们可以在每个时间步使用相同参数的相同转移函数 $f$

### 循环神经网络的结构

基于上一节的**图展开**和**参数共享**的思想，我们可以继续学习现在最常用的 RNN 形式。

实际上 RNN 的具体形式或者构造种类非常多，这里我们只介绍最常见的 RNN 形式，其他的部分 RNN 形式可以参考《深度学习》 10.2 节“循环神经网络”。

下图是当前最具有代表性的 RNN 例子：

![RNN2](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/RNN2.png?raw=true)

其中：

- 损失 $L$ 衡量每个输出 $o$ 与相应训练目标真实值 $y$ 的距离；
- RNN 输入到隐藏的连接由权重矩阵 $U$ 参数化；
- 隐藏到隐藏的循环连接由权重矩阵 $W$ 参数化；
- 隐藏到输出的连接由权重矩阵 $V$ 参数化。

现在我们来研究一下这个图中 RNN 的前向传播公式。

首先，此图中没有指定隐藏单元的**激活函数**，我们假定它用的是双曲正切函数作为激活函数。

其次，此图中没有指定**输出**和**损失函数**，我们假定输出是离散的，比如这是一个用于预测字符、词语的 RNN。

那么，表现离散变量的常用方式（回想一下分类任务是怎么做的），就是把输出 $o$ 作为每个离散变量可能值的非标准化对数概率。然后，我们可以应用 `softmax` 函数处理，获得标准化后概率的输出向量 $\hat y$。

RNN 从特定的初始状态 $h^{(0)}$ 开始前向传播。从 $t=1$ 到 $t=\tau$ 的每个时间步，我们有如下更新方程：

$$
\large 
\begin{equation}\begin{split} 
a^{(t)} &= b + Wh^{(t-1)} + Ux^{(t)} \\
h^{(t)} &= \tanh (a^{(t)})\\
o^{(t)} &= c + Vh^{(t)}\\
\hat y^{(t)} &= {\rm softmax}(o^{(t)})\\
\end{split} \end{equation}$$

刚刚开始看方程可能会有些困难，所以我们一步一步，利用图片来理解。

首先是 $a^{(t)} = b + Wh^{(t-1)} + Ux^{(t)}$，它在上面的大图中所在的过程如下图所示。

![RNN3-2](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/RNN3-2.png?raw=true)

经过上图的运算我们得到了 $a^{(t)}$。注意，这里新增了偏置 $b$ 等细节，比之前的整体展开图更加详细。

接下来是 $h^{(t)} = \tanh (a^{(t)})$，过程如下：

![RNN4](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/RNN4.png?raw=true)

仅仅是对 $a^{(t)}$ 应用了一个激活函数，非常简单的一步。

第三步是 $o^{(t)} = c + Vh^{(t)}$，过程如下：

![RNN5](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/RNN5-1.png?raw=true)

其中 $c$ 也是偏置向量。到现在我们得到了输出 $o^{(t)}$。

至于第四步 $\hat y^{(t)} = {\rm softmax}(o^{(t)})$ 的过程，其实是在损失函数 $L$ 内部计算的，然后会与真实标签 $y^{(t)}$ 相比较，这里就略去不画了。

到这里我们就大致了解了循环神经网络的前向传播过程。

> 在这种 RNN 结构中，由于前向传播图是固有顺序的，每个时间步只能一前一后地计算，而且前向传播过程中各个状态 $h$ 必须保存，用以在反向传播中再次使用，所以其运行时间复杂度为 $O(\tau)$，而空间复杂度也是 $O(\tau)$。

> 关于循环神经网络的反向传播推导可参考《深度学习》第10.2.2 节 “计算循环神经网络的梯度”。

## 运用实例说明RNN



## 循环神经网络的编程实现

